In [1]:
# 1. INSTALL & IMPORT LIBRARY
!pip install -q plotly ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.0 MB/s eta 0:00:00


In [2]:
# 2. UPLOAD DATASET
from google.colab import files
print("forecasting_dataset.csv")
uploaded = files.upload()

forecasting_dataset.csv


Saving forecasting_dataset.csv to forecasting_dataset.csv


In [3]:
# 3. BACA DATA & RENAME KOLOM
df = pd.read_csv("forecasting_dataset.csv")
df = df.rename(columns={
    "Irradiance (kWh/m²)": "Irradiance",
    "Temperature (°C)": "Temperature",
    "Humidity (%)": "Humidity",
    "Solar Output (MWh)": "Solar Output"
})

In [4]:
# 4. PARSE DATE & BUAT DAY NUMBER
df["Date"] = pd.to_datetime(df["Date"])
df["Day_Number"] = (df["Date"] - df["Date"].min()).dt.days + 1

print("Data berhasil dibaca!")
print(df.head())

Data berhasil dibaca!
        Date  Irradiance  Temperature   Humidity  Solar Output  Day_Number
0 2023-01-01    5.745071    27.169259  63.577874      0.983215           1
1 2023-01-02    4.792604    29.158709  65.607845      0.846485           2
2 2023-01-03    5.971533    29.314571  70.830512      1.343765           3
3 2023-01-04    7.284545    28.395445  70.538021      1.578983           4
4 2023-01-05    4.648770    29.677429  46.223306      0.925574           5


In [6]:
# 5. REGRESI MULTIVARIAT (Irradiance, Temperature, Humidity → Solar Output)
X = df[["Irradiance", "Temperature", "Humidity"]]
X = sm.add_constant(X)
y = df["Solar Output"]

model = sm.OLS(y, X).fit()
df["Predicted"] = model.predict(X)
df["Error"] = df["Predicted"] - df["Solar Output"]

print("\n HASIL REGRESI MULTIVARIAT")
print(model.summary())


 HASIL REGRESI MULTIVARIAT
                            OLS Regression Results                            
Dep. Variable:           Solar Output   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     62.17
Date:                Wed, 17 Sep 2025   Prob (F-statistic):           2.03e-22
Time:                        00:12:17   Log-Likelihood:                 33.483
No. Observations:                 100   AIC:                            -58.97
Df Residuals:                      96   BIC:                            -48.54
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.2182

In [7]:
# 6. HITUNG METRIK EVALUASI
mae = mean_absolute_error(df["Solar Output"], df["Predicted"])
rmse = np.sqrt(mean_squared_error(df["Solar Output"], df["Predicted"]))
mape = np.mean(np.abs(df["Error"] / df["Solar Output"])) * 100

print(f"\n Evaluasi Model:")
print(f"MAE : {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"MAPE: {mape:.2f}%")


 Evaluasi Model:
MAE : 0.142
RMSE: 0.173
MAPE: 16.43%


In [8]:
# 7. DASHBOARD INTERAKTIF (DROPDOWN)
print("\n Dashboard Interaktif - Pilih Tanggal untuk Lihat Hasil Prediksi")

dropdown = widgets.Dropdown(
    options=["All"] + list(df["Date"].dt.strftime("%Y-%m-%d").unique()),
    description="Tanggal:"
)

def update_dashboard(selected_date):
    if selected_date != "All":
        filtered_df = df[df["Date"].dt.strftime("%Y-%m-%d") == selected_date]
    else:
        filtered_df = df

    # Scatter Plot: Prediksi vs Aktual
    fig1 = px.scatter(
        filtered_df,
        x="Solar Output",
        y="Predicted",
        title="Prediksi vs Aktual Solar Output",
        labels={"Solar Output": "Actual", "Predicted": "Predicted"},
        trendline="ols"
    )
    fig1.show()

    # Line Chart: Actual vs Predicted per Date
    fig2 = px.line(
        filtered_df,
        x="Date",
        y=["Solar Output", "Predicted"],
        title="Time Series: Aktual vs Prediksi"
    )
    fig2.show()

try:
    widgets.interact(update_dashboard, selected_date=dropdown)
except:
    print("Widget tidak didukung di Colab. Menampilkan semua grafik sekaligus...")
    update_dashboard("All")


 Dashboard Interaktif - Pilih Tanggal untuk Lihat Hasil Prediksi


interactive(children=(Dropdown(description='Tanggal:', options=('All', '2023-01-01', '2023-01-02', '2023-01-03…

# Interpretasi Hasil

## 1. Hasil Regresi Multivariat
- **R-squared = 0.660** → artinya 66% variasi Solar Output dapat dijelaskan oleh kombinasi Irradiance, Temperature, dan Humidity.
- **Koefisien terbesar adalah Irradiance (0.1764)** → ini berarti setiap kenaikan 1 kWh/m² irradiance akan meningkatkan Solar Output sebesar ±0.176 MWh.
- **P-value Irradiance = 0.000 (signifikan)** → pengaruhnya nyata terhadap Solar Output.
- **Temperature & Humidity memiliki p-value > 0.05** → artinya kontribusinya terhadap model tidak signifikan secara statistik.
- Adjusted R² (0.650) cukup tinggi → model cukup baik untuk menjelaskan data.

## 2. Evaluasi Model
| Metrik | Nilai |
|-------|-------|
| **MAE** | 0.142 |
| **RMSE** | 0.173 |
| **MAPE** | 16.43% |

Interpretasi:
- MAE & RMSE kecil → selisih rata-rata prediksi vs aktual rendah.
- MAPE = 16.43% → model memiliki akurasi sekitar 83.5% (cukup baik, walau bisa ditingkatkan).

## 3. Visualisasi Dashboard
- **Scatter Plot** menunjukkan bahwa sebagian besar titik mendekati garis trendline → prediksi model cukup mendekati nilai aktual.
- **Time Series Chart** memperlihatkan pola Predicted vs Actual hampir sejajar → artinya model mengikuti tren data dengan baik.

## 4. Kesimpulan
- Variabel paling penting dalam memprediksi Solar Output adalah **Irradiance**.
- Model multivariat meningkatkan akurasi dibanding hanya pakai satu variabel.
- Dashboard membantu memudahkan analisis interaktif: user bisa memfilter tanggal dan melihat perbandingan hasil prediksi vs aktual.